### 직방 서비스 원룸 데이터 수집
- 복잡한 데이터 수집 방법
- 동이름 > 매물정보(데이터프레임)

#### [절차]
- 동이름 > 위도, 경도
- 위도 , 경도 > geohash code(지역범위)
- geohash > 매물 아이디
- 매물 아이디 > 매물 정보

In [85]:
import pandas as pd
import requests

In [86]:
# 1. 동이름 > 위도, 경도

In [87]:
query = "망원동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={query}&serviceType=원룸"
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.556785583496094, 126.9013442993164)

In [88]:
# 2. 위도, 경도 > geohash

In [89]:
# !pip install geohash2

In [90]:
import geohash2

In [108]:
# precision : 영역 : 값이 커질수록 영역이 작아짐(값이 커진다는 것은 더 세세하게 보겠다는 의미이므로 영역이 줄어듦으로 이해 )
code = geohash2.encode(lat, lng, precision=5)

In [109]:
# 3. geohash > 매물 아이디

In [110]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
response = requests.get(url)
response

<Response [200]>

In [111]:
response.text

'{"clusters":[],"items":[{"lat":37.529599547169035,"lng":126.89759004056587,"item_id":30462357},{"lat":37.529112660327144,"lng":126.89697283518173,"item_id":30524930},{"lat":37.52910290121822,"lng":126.89698203743734,"item_id":30629925},{"lat":37.52911700113404,"lng":126.89700703284934,"item_id":30681505},{"lat":37.52909892224272,"lng":126.89134810739006,"item_id":30494154},{"lat":37.52931506982884,"lng":126.8973052757723,"item_id":30351843},{"lat":37.52931219055722,"lng":126.89792183902809,"item_id":30620037},{"lat":37.52988265275184,"lng":126.90723352912393,"item_id":30218399},{"lat":37.53107561693497,"lng":126.90695890144072,"item_id":30673258},{"lat":37.53059358938474,"lng":126.9069825317213,"item_id":30673399},{"lat":37.530770993576645,"lng":126.90756873908897,"item_id":30355099},{"lat":37.53127320079945,"lng":126.90697851801424,"item_id":30444123},{"lat":37.531244256866856,"lng":126.9076045440421,"item_id":30466344},{"lat":37.5312843284334,"lng":126.90759604102428,"item_id":30560

In [112]:
data = response.json()["items"]
data[:3]

[{'lat': 37.529599547169035, 'lng': 126.89759004056587, 'item_id': 30462357},
 {'lat': 37.529112660327144, 'lng': 126.89697283518173, 'item_id': 30524930},
 {'lat': 37.52910290121822, 'lng': 126.89698203743734, 'item_id': 30629925}]

In [103]:
# sol1)
# ids = []
# for data in data:
#     ids.append(data["item_id"])
# ids[:5]

In [113]:
# list comprehention
ids = [data["item_id"] for data in data]
ids[:3]

[30462357, 30524930, 30629925]

In [105]:
# # list comprehention : 간단한 리스트 데이터를 만들때
# # 홀수 데이터만 제곱해서 리스트 만들어 출력
# result = [data**2 for data in range(10) if data % 2]
# result

In [106]:
# 4. 매물 아이디 > 매물 정보

In [126]:
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain":"zigbang",
    "withCoalition":"true",
    "item_ids":ids[:998],
}

response = requests.post(url, params)
response

<Response [200]>

response.text

In [127]:
items = response.json()["items"]
item_df = pd.DataFrame(items)
item_df.tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
906,None,30455640,https://ic.zigbang.com/ic/items/30455640/1.jpg,전세,전세,28000,0,33.58,"{'m2': 33.58, 'p': '10.2'}","{'m2': 23.98, 'p': '7.3'}",None,None,6,6,6,🔶🔶마포 저렴하고 전망 좋은 복층 쓰리룸 탑층🔶🔶,None,05,마포구 중동,"{'lat': 37.56946132977145, 'lng': 126.90885427...",False,True,빌라,[추천],서울시 마포구 중동,None,None,3,2022-02-03T12:00:40+09:00,False
907,None,30687168,https://ic.zigbang.com/ic/items/30687168/1.jpg,월세,월세,5000,60,41.36,"{'m2': 41.36, 'p': '12.5'}","{'m2': 36.36, 'p': '11'}",None,None,3,3,7,🟥🟨🟩 깔끔한 분위기~! 넓은 신축형 복층 원룸,None,04,마포구 중동,"{'lat': 37.56996065473448, 'lng': 126.90883976...",False,True,빌라,[],서울시 마포구 중동,None,None,8,2022-02-18T14:00:36+09:00,False


In [128]:
# 모든 데이터 보기
pd.options.display.max_columns = 30
pd.options.display.max_rows = 60

In [129]:
item_df.columns

Index(['section_type', 'item_id', 'images_thumbnail', 'sales_type',
       'sales_title', 'deposit', 'rent', 'size_m2', '공급면적', '전용면적', '계약면적',
       'room_type_title', 'floor', 'floor_string', 'building_floor', 'title',
       'is_first_movein', 'room_type', 'address', 'random_location', 'is_zzim',
       'status', 'service_type', 'tags', 'address1', 'address2', 'address3',
       'manage_cost', 'reg_date', 'is_new'],
      dtype='object')

In [130]:
columns = ['item_id', 'sales_title', 'deposit', 'rent', 'size_m2',
           'floor', 'building_floor', 'title','room_type', 'address',
           'service_type','address1', 'manage_cost', 'reg_date', 'is_new',
]

In [131]:
result_df = item_df[columns]
# geohash값으로 인해 다른 구의 데이터가 나오는 것을 방지 하기 위한 방법
# 마포구 합정동이라는 결과가 들어있는지 아닌지에 따라 true/false로 나옴
result_df = result_df[result_df["address"].str.contains(query)]
# 인덱스 정렬을 시켜주기 위해서
result_df = result_df.reset_index(drop=True)
result_df.tail(1)

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
94,30684919,월세,200,60,19.83,2,3,✔마포구청역 도보4분. 신축급 원룸 월세,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-17T18:11:46+09:00,False


In [132]:
# 보증금 1억 이하, 월세 100만원 이하
result_df[(result_df["deposit"] <= 10000) & (result_df["rent"] <= 100)]

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
4,30340227,월세,1000,49,23.14,3,5,⭕망원역세권 7평원룸 내부깔끔⭕,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-01-22T14:38:50+09:00,False
5,30691402,월세,5000,90,33.06,4,4,🌈여성1인전용📢대출가능👑망원역6분🌈테라스💖,02,마포구 망원동,원룸,서울시 마포구 망원동,10,2022-02-18T12:12:34+09:00,False
6,30604487,전세,9300,0,15.00,2,5,망리단길 인접 풀옵션 원룸,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-02-21T02:39:16+09:00,True
11,30734072,월세,2000,70,33.06,3,4,⭕망원역세권 분리형원룸 10평 채광좋음⭕,02,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-02-21T16:23:40+09:00,True
16,30687981,월세,1000,85,49.59,3,4,⭕망원역세권 남향투룸 내부깔끔⭕,04,마포구 망원동,빌라,서울시 마포구 망원동,5,2022-02-18T09:50:36+09:00,False
17,30488407,월세,500,43,19.83,옥탑방,3,🤩이넓은테라스가다내꺼🌞,02,마포구 망원동,원룸,서울시 마포구 망원동,2,2022-02-16T17:10:20+09:00,False
21,30620775,월세,3000,100,52.89,3,5,⭕망원동 한강공원인근 투룸월세⭕,04,마포구 망원동,빌라,서울시 마포구 망원동,2,2022-02-14T10:25:58+09:00,False
27,30592602,월세,2000,55,16.53,3,5,🎉이쁜 인테리어 풀옵 원룸🎉,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-15T12:53:03+09:00,False
31,30522407,월세,10000,60,42.84,2,5,중기청80% 통베란다 넓고 전대가능한 투룸 월세,04,마포구 망원동,빌라,서울시 마포구 망원동,3,2022-02-07T14:39:54+09:00,False
35,30707009,월세,3000,100,59.50,2,3,⭕망원시장안 쓰리룸 월세 주거용사무실가능⭕,05,마포구 망원동,빌라,서울시 마포구 망원동,0,2022-02-19T13:02:03+09:00,False


In [133]:
# 5. 함수 만들기

In [134]:
def oneroom(address):
        
        # 1. 동이름 > 위도, 경도
        url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸"
        response = requests.get(url)
        data = response.json()["items"][0]
        lat, lng = data["lat"], data["lng"]

        # 2. 위도, 경도 > geohash
        code = geohash2.encode(lat, lng, precision=5)

        # 3. geohash > 매물 아이디
        url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={code}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
        response = requests.get(url)
        data = response.json()["items"]
        ids = [data["item_id"] for data in data]

        # 4. 매물 아이디 > 매물 정보
        url = "https://apis.zigbang.com/v2/items/list"
        params = {"domain":"zigbang",
                  "withCoalition":"true",
                  "item_ids":ids[:998]
        }
        response = requests.post(url, params)
        items = response.json()["items"]
        item_df = pd.DataFrame(items)
        
        columns = ['item_id', 'sales_title', 'deposit', 'rent', 'size_m2',
               'floor', 'building_floor', 'title','room_type', 'address',
               'service_type','address1', 'manage_cost', 'reg_date', 'is_new',
        ]
        
        result_df = item_df[columns]
        # geohash값으로 인해 다른 구의 데이터가 나오는 것을 방지 하기 위한 방법
        # 마포구 합정동이라는 결과가 들어있는지 아닌지에 따라 true/false로 나옴
        result_df = result_df[result_df["address"].str.contains(address)]
        # 인덱스 정렬을 시켜주기 위해서
        result_df = result_df.reset_index(drop=True)
        
        return result_df

In [135]:
oneroom("망원동")

,item_id,sales_title,deposit,rent,size_m2,floor,building_floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
0,30483493,전세,23000,0,18.99,6,6,"🦋합정역,신축1.5룸가성비짱,한강공원도보,보증보험OK",02,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-04T15:40:23+09:00,False
1,30608827,전세,23000,0,42.98,6,6,방1개 별도거실 주방 화장실 다용도실 분리형원룸 입니다,04,마포구 망원동,빌라,서울시 마포구 망원동,7,2022-02-12T16:27:27+09:00,False
2,30578522,월세,12000,10,12.21,3,5,💥LH 가능 깨끗한 방💥,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-15T18:54:14+09:00,False
3,30726074,월세,12000,10,13.22,3,5,🎶 [LH 가능!] 🎶 [깔끔한 방!],01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-21T11:45:06+09:00,True
4,30340227,월세,1000,49,23.14,3,5,⭕망원역세권 7평원룸 내부깔끔⭕,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-01-22T14:38:50+09:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,30573911,전세,37000,0,36.17,2,6,마포구청역 3분.전세대출가능.무융자.신축급 컨디션 투룸~!,04,마포구 망원동,빌라,서울시 마포구 망원동,4.3,2022-02-10T13:55:31+09:00,False
91,30606067,전세,37000,0,39.67,2,6,🌷 대출가능! 투룸! 깔끔한 인테리어! 6호선! 🌷,04,마포구 망원동,빌라,서울시 마포구 망원동,5,2022-02-16T18:36:13+09:00,False
92,30713445,월세,5000,30,19.83,3,4,🎢마포구청역5분 신축3년차🎢,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-21T13:46:30+09:00,True
93,30594723,월세,200,55,16.53,1,3,💖1.5층 단기방! 마포구청역 3분!💖,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-14T09:49:15+09:00,False
